In [1]:
#Andrii comments are [A: ]
#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#We work with lattices D_n, n>=3, 
#All parameters (basis, covering radius) are from Conway Sloan 'Sphere packings, lattices and Groups', 3ed, p.117
n=4 #n--lattice dim
m=n #m--space dim
L = [[1,1]+[0]*(n-2)] #L -- a basis matrix for the lattice, rows are basis vectors
for i in range(n-1):
    L.append([0]*i+[1,-1]+[0]*(n-2-i))

lat_det=abs(matrix(L).det())    
    
#covering radius, 
R=1 #if n=3
if n>3:
    R=sqrt(n)/2


#reduced form of a vector x: non-increasing, non-negative
def reduce(x):
    y = [abs(t) for t in x]
    y.sort(key=lambda t: -t) 
    return(y)

# first we generate all nodes of a lattice with norm <=4R
# Since Automorphism group contains all coordinate permutations, we only generate those nodes that 
# have non-increasing non-negative coordinates
short = []
K = ceil(4*R)
K2 = K^2  # [A: Maybe K2=16R^2, slightly smaller?]
for x in mrange_iter([range(K+1) for i in range(n)]):
    descending = True
    for i in range(n-1):
        if x[i]<x[i+1]:
            descending = False
            break
    if descending and sum(x)%2==0 and sum(t^2 for t in x)<=K2 and x[0]>0:
        short.append(list(x))

print("short:",len(short))


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = []
for i in range(n-1):
    toadd = [0]*(n+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd)
ineqs.append([0]*n+[1])

#now Voronoi cell planes
for p in short:
    ineqs.append([sum((p[i])^2 for i in range(n))]+[-2*p[i] for i in range(n)])

V = Polyhedron(ieqs=ineqs)
    
### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))


### forbd = forb with possible permutations and signs
forbd = {}
forbdv = []
for x in forb:
    for k in mrange_iter([[-1,1] for i in range(n)]): #sign changes
        y = [x[j]*k[j] for j in range(n)]
        for t in Permutations(y):
            vt = vector(t)
            if str(vt) not in forbd:
                forbd[str(vt)]=0
                if sum(t)>=0:
                    forbdv.append(vt)
print("forbidden without symmetries:",len(forbd))
###

fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
fn = sqrt(fn)
print("L2 norm of forbidden:",fn,fn.n())


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) 
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    A = A.transpose() #now columns

    Ad = abs(A.det()/lat_det)
    if Ad==0 or Ad>=target:
        return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

        
    Ai = A.inverse() #if x=Ai f, we want to estimate the largest value of x_i using Cauchy-Schwartz
    Ai_norms = [Ai[i].norm() for i in range(n)] #norm of the row of the inverse |x_i|\le Ai_norms[i]*fn
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(fn*Ai_norms[i])
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return Ad


if n==4:
    print(check_sublattice([[-1, 0, 3, -2], [3, -1, 0, 2], [-2, 0, -1, 3], [-2, 3, 0, 1]]))   



short: 12
forbidden: 9
forbidden without symmetries: 408
L2 norm of forbidden: 2*sqrt(3) 3.46410161513775
small: 354
49
